In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pwd

/content


In [0]:
import os
os.chdir("/")

In [0]:
!pwd

/


In [0]:
!ls

bin					   etc	  opt	 sys
boot					   home   proc	 tensorflow-2.1.0
content					   lib	  root	 tmp
datalab					   lib32  run	 tools
dev					   lib64  sbin	 usr
dlib-19.18.0-cp27-cp27mu-linux_x86_64.whl  media  srv	 var
dlib-19.18.0-cp36-cp36m-linux_x86_64.whl   mnt	  swift


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/spark-2.4.1-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
spark

In [0]:
!jps

513 Jps
421 SparkSubmit


In [0]:
from pyspark import SparkContext
sc = spark.sparkContext

**Naive Bayes**

Referensi: https://runawayhorse001.github.io/LearningApacheSpark/classification.html

1. Menyiapkan spark context dan SparkSession

In [0]:
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .appName("PySpark Naive Bayes Classifier") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

2. Load dataset

In [0]:
df = spark.read.format('com.databricks.spark.csv') \
            .options(header='true', inferschema='true') \
            .load("nursery.csv",header=True);

In [0]:
df.show(5)

+-------+--------+--------+--------+----------+----------+-------------+-----------+---------+
|parents|has_nurs|    form|children|   housing|   finance|       social|     health|    class|
+-------+--------+--------+--------+----------+----------+-------------+-----------+---------+
|  usual|  proper|complete|     one|convenient|convenient|      nonprob|recommended|recommend|
|  usual|  proper|complete|     one|convenient|convenient|      nonprob|   priority| priority|
|  usual|  proper|complete|     one|convenient|convenient|      nonprob|  not_recom|not_recom|
|  usual|  proper|complete|     one|convenient|convenient|slightly_prob|recommended|recommend|
|  usual|  proper|complete|     one|convenient|convenient|slightly_prob|   priority| priority|
+-------+--------+--------+--------+----------+----------+-------------+-----------+---------+
only showing top 5 rows



3. Fungsi untuk mengubah fitur kategorikal menjadi numerik dan disimpan dalam bentuk *Dense Vector*

In [0]:
def get_dummy(df,categoricalCols,continuousCols,labelCol):

    from pyspark.ml import Pipeline
    from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
    from pyspark.sql.functions import col

    indexers = [ StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c))
                 for c in categoricalCols ]

    # default setting: dropLast=True
    encoders = [ OneHotEncoder(inputCol=indexer.getOutputCol(),
                 outputCol="{0}_encoded".format(indexer.getOutputCol()))
                 for indexer in indexers ]

    assembler = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders]
                                + continuousCols, outputCol="features")

    pipeline = Pipeline(stages=indexers + encoders + [assembler])

    model=pipeline.fit(df)
    data = model.transform(df)

    data = data.withColumn('label',col(labelCol))

    return data.select('features','label')

In [0]:
categorical = ['parents','has_nurs','form','children','housing','finance','social','health']
numeric = []
class_output = 'class'

In [0]:
data = get_dummy(df,categorical,numeric,class_output)
data.show(5)

+--------------------+---------+
|            features|    label|
+--------------------+---------+
|(19,[1,4,7,10,18]...|recommend|
|(19,[1,4,7,10,17]...| priority|
|(19,[1,4,7,10],[1...|not_recom|
|(19,[1,4,7,10,16,...|recommend|
|(19,[1,4,7,10,16,...| priority|
+--------------------+---------+
only showing top 5 rows



4. merubah label kategorikal

In [0]:
from pyspark.ml.feature import StringIndexer
# Index labels, adding metadata to the label column
labelIndexer = StringIndexer(inputCol='label',
                             outputCol='indexedLabel').fit(data)
labelIndexer.transform(data).show(5, True)

+--------------------+---------+------------+
|            features|    label|indexedLabel|
+--------------------+---------+------------+
|(19,[1,4,7,10,18]...|recommend|         4.0|
|(19,[1,4,7,10,17]...| priority|         1.0|
|(19,[1,4,7,10],[1...|not_recom|         0.0|
|(19,[1,4,7,10,16,...|recommend|         4.0|
|(19,[1,4,7,10,16,...| priority|         1.0|
+--------------------+---------+------------+
only showing top 5 rows



5. fitur yang memiliki *unique value* lebih dari 4 akan dianggap sebagai fitur kontinu

In [0]:
from pyspark.ml.feature import VectorIndexer
# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", \
                                  outputCol="indexedFeatures", \
                                  maxCategories=5).fit(data)
featureIndexer.transform(data).show(5, True)

+--------------------+---------+--------------------+
|            features|    label|     indexedFeatures|
+--------------------+---------+--------------------+
|(19,[1,4,7,10,18]...|recommend|(19,[1,4,7,10,18]...|
|(19,[1,4,7,10,17]...| priority|(19,[1,4,7,10,17]...|
|(19,[1,4,7,10],[1...|not_recom|(19,[1,4,7,10],[1...|
|(19,[1,4,7,10,16,...|recommend|(19,[1,4,7,10,16,...|
|(19,[1,4,7,10,16,...| priority|(19,[1,4,7,10,16,...|
+--------------------+---------+--------------------+
only showing top 5 rows



6. split data training:data test dengan perbandingan 60:40

In [0]:
# Split the data into training and test sets (40% held out for testing)
(trainingData, testData) = data.randomSplit([0.6, 0.4])

trainingData.show(5,False)
testData.show(5,False)

+----------------------------------------------+---------+
|features                                      |label    |
+----------------------------------------------+---------+
|(19,[],[])                                    |not_recom|
|(19,[0],[1.0])                                |not_recom|
|(19,[0,2],[1.0,1.0])                          |not_recom|
|(19,[0,2,6,9,12],[1.0,1.0,1.0,1.0,1.0])       |not_recom|
|(19,[0,2,6,9,12,14],[1.0,1.0,1.0,1.0,1.0,1.0])|not_recom|
+----------------------------------------------+---------+
only showing top 5 rows

+------------------------------------------------------------+----------+
|features                                                    |label     |
+------------------------------------------------------------+----------+
|(19,[0,2,6],[1.0,1.0,1.0])                                  |not_recom |
|(19,[0,2,6,9],[1.0,1.0,1.0,1.0])                            |not_recom |
|(19,[0,2,6,9,12,14,15],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])       |not_recom |


In [0]:
from pyspark.ml.linalg import Vectors # !!!!caution: not from pyspark.mllib.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.feature import IndexToString,StringIndexer, VectorIndexer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

7. fit Naive Bayes model

In [0]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(featuresCol='indexedFeatures', labelCol='indexedLabel')

8. Arsitektur *pipeline*

In [0]:
# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

In [0]:
# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, nb,labelConverter])

In [0]:
# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

9. Membuat prediksi menggunakan data uji

In [0]:
# Make predictions.
predictions = model.transform(testData)
# Select example rows to display.
predictions.select("features","label","predictedLabel").show(5)

+--------------------+----------+--------------+
|            features|     label|predictedLabel|
+--------------------+----------+--------------+
|(19,[0,2,6],[1.0,...| not_recom|    spec_prior|
|(19,[0,2,6,9],[1....| not_recom|     not_recom|
|(19,[0,2,6,9,12,1...| not_recom|     not_recom|
|(19,[0,2,6,9,12,1...|spec_prior|    spec_prior|
|(19,[0,2,6,9,12,1...|spec_prior|    spec_prior|
+--------------------+----------+--------------+
only showing top 5 rows



10. evaluasi

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.152129
